# Einführung Multi-Variate Datenanalyse (MVDA)

## Übung 1: Visualisierung hoch-dimensionaler Datensätze mit Python
für Kurse Mathematische Berechnungssoftware (MBS)

_Simeon Sauer, SS2020_

#### Bibliotheken laden

In [4]:
# wie im Video besprochen, müssen wir die Bibliotheken "pandas" und "plotly" laden
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go

#### Daten importieren

Wir importieren diesmal einen Datensatz, in dem Weizenkörner für verschiedende Sorten gemeoterisch vermessen wurden.
Mehr Dateils hier: http://archive.ics.uci.edu/ml/datasets/seeds

- Zeilen: Unterschiedliche Proben von Saatkörner
- Spalten: 7 - Weizensorte; 0-6: verschiedene Messwerte


In [10]:
data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt", sep='\t+', header=None)
data

/Users/sauers/opt/anaconda3/envs/lectures/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [13]:
# Spaltenname einführen:
data.columns = [
    "area A",
    "perimeter P",
    "compactness C",
    "length of kernel",
    "width of kernel",
    "asymmetry coefficient",
    "length of kernel groove",
    "type"]

# Index-Spalte wird separat behandelt in pandas-Bibliothek
data.index.name='sample no.'

# Jetzt nennen wir noch die Weizensorte um
Weizensorten = ['Kama', 'Rosa', 'Canadian']
data['type'] = data['type'].replace([1,2,3], Weizensorten)

# Wie sieht die Datentabelle jetzt aus?
data

,area A,perimeter P,compactness C,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,type
sample no.,,,,,,,,
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,Kama
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,Kama
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,Kama
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,Kama
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,Kama
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,Canadian
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,Canadian
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,Canadian


#### slicing

In [17]:
# wählen Sie Beobachtungen aus, in denen die Fläche A>15 ist, und der Asymmetry-Koeffizient < 1.5
slice = (data['area A']>15) & (data['asymmetry coefficient'] < 1.5)
data[slice]

,area A,perimeter P,compactness C,length of kernel,width of kernel,asymmetry coefficient,length of kernel groove,type
sample no.,,,,,,,,
4,16.14,14.99,0.9034,5.658,3.562,1.3550,5.175,Kama
22,15.88,14.90,0.8988,5.618,3.507,0.7651,5.091,Kama
25,16.19,15.16,0.8849,5.833,3.421,0.9030,5.307,Kama
46,15.36,14.76,0.8861,5.701,3.393,1.3670,5.132,Kama
83,19.57,16.74,0.8779,6.384,3.772,1.4720,6.273,Rosa


In [18]:
# Sie sollten fünf Zeilen (=5 Körner) gefunden haben. Was ist der mittlere Durchmesser für diese 5 Körner?
data[slice].mean()

area A                     16.62800
perimeter P                15.31000
compactness C               0.89022
length of kernel            5.83880
width of kernel             3.53100
asymmetry coefficient       1.17242
length of kernel groove     5.39560
dtype: float64

## Explorative Datenanalyse (EDA)

In [22]:
# Setzen Sie in untenstehendem Code als Variable ('x') den Umfang ('Perimeter P') ein, und als Farbe ('color') die Weizensort ('type')
px.histogram(data,
                   x="perimeter P",
                   color="type", # einzelne Histogramme für jede Weinsorte
                   nbins=20, # Anzahl Balken ('bins')
                   barmode='group', # gruppiere die Balken für die 3 Weinsorten, nicht überlagern
                   marginal='box', # Boxplot über das Histogram
                   histnorm='percent') # normieren Histogramm auf 100%, nicht auf die Anzahl der Beobachtung

#### Fragen zum Histogram:
* Kann man die drei Sorten anhand des Umfangs eindeutig trennen?
* Oder gibt es wenigstens zwei Sorten, die sich trennen lassen?

_Rosa und Canadian lassen sich trennen, aber nicht alle drei Sorten._

#### scatter matrix plot

In [32]:
# Färben Sie im Scatter Matrix Plot die Datenpunkte anhand ihrer Sorte ('type') ein.
px.scatter_matrix(data,
                  color='type',
                  dimensions=data.iloc[:,0:7],
                  opacity=0.8, # macht Datenpunkte zu 20% durchsichtig
                  height=1000, # Höhe des Gesamtplots in Pixeln
                  hover_name=data.index) # Überschrift der eingeblendeten Mouse-Over-Information: Zeilennr.

Frage: Welche Variablen sind stark miteinander korreliert?

_z.B. 'Area A' und 'Perimenter P'; klar, großer Umfang geht auch mit großer Fläche einher._

#### Bubble chart

In [38]:
# a. wählen Sie anhand des Scatter Matrix Plots 2 Variablen aus, die die 3 Sorten am besten trennen
# b. tragen Sie diese Variable als 'x' und 'y' im untenstehenden bubble chart ein.
# c. Wählen Sie für die Punktgröße ('size') eine beliebige dritte Variable

px.scatter(data,
           x="perimeter P",
           y="length of kernel groove",
           size="asymmetry coefficient",
           color="type",
           hover_name=data.index,
           size_max=30)